<a href="https://colab.research.google.com/github/BoraShruti/Duplicate-Quota-Questions/blob/main/Duplicate_Qoura_%2B_fe_%2B_dask_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import dask.dataframe as dd
!python -m pip install "dask[distributed]" --upgrade
!pip install dask-ml
from dask.distributed import Client, progress # importing Client and progress
from dask_ml.feature_extraction.text import HashingVectorizer
from dask_ml.wrappers import Incremental
from dask_ml.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier


In [ ]:
!pip install dask-tfidf
#from dask_ml.feature_extraction.text import TfidfTransformer


In [ ]:
# setting up Dask Client
csvfile = "/content/drive/MyDrive/train.csv"
client = Client(n_workers=4)
df = dd.read_csv(csvfile)
df.shape

In [ ]:
# @title
import csv
csvfile = "/content/drive/MyDrive/train.csv"
df = pd.read_csv(csvfile)
df.shape

In [ ]:
df.head()
#df = df.rename(columns = {"8200\"":"id" , "96804": "qid1", "177839": "qid2", "How do I known that someone viewed my Facebook profile?":"q1", "How do I know viewers of my Facebook profile?":"q2", "1": "label"})

id: not of much use


In [ ]:
df = df.sample(30000,random_state=2)
df.info()

In [ ]:
# find missing values
df.isnull().sum() # not much data missing

In [ ]:
# counting rows that are completely duplicate duplicate
df.duplicated().sum()

In [ ]:
#distribution of duplicate and non-duplicate questions
print(df['is_duplicate'].value_counts())
print((df['is_duplicate'].value_counts()/df['is_duplicate'].count())*100)
df['is_duplicate'].value_counts().plot(kind='bar')

#biased dataset more samples of non duplicate

In [ ]:
qid = pd.Series(df['qid1'].tolist() + df['qid2'].tolist())
print('Count of unique questions', np.unique(qid).shape[0])
x = qid.value_counts()>1
print('Number of questions getting repeated', x[x].shape[0])

In [ ]:
# repeat ques histogram
plt.hist(qid.value_counts().values, bins = 160)
plt.yscale('log')
plt.show()

In [ ]:
def common_words(row):
  #
  w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" "))) # set removes duplicate words
  w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
  return len(w1 & w2) # intersection between the list of words in both questions
def total_words(row):
  #
  w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" "))) # set removes duplicate words
  w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))
  return (len(w1) + len(w2)) # intersection between the list of words in both questions

In [ ]:
# Feature Engineering
df['q1_len'] = df['question1'].str.len()
df['q2_len'] = df['question2'].str.len()
df['q1_len_w'] = df['question1'].apply(lambda row: len(row.split(" ")))
df['q2_len_w'] = df['question2'].apply(lambda row: len(row.split(" ")))
df['word_common'] = df.apply(common_words, axis=1)
df['total_words'] = df.apply(total_words,axis=1)
df['word_share'] = round(df['word_common']/df['total_words'],2) # roundoff to 2 decimal places
df.head()

In [ ]:
# Analysis of features - q1_len
sns.displot(df['q1_len'])
print("max of q1_len",df['q1_len'].max())
print("min of q1_len",df['q1_len'].min())
print("mean of q1_len",df['q1_len'].mean())

In [ ]:
# Analysis of features - q2_len
sns.displot(df['q2_len'])
print("max of q2_len",df['q2_len'].max())
print("min of q2_len",df['q2_len'].min())
print("mean of q2_len",df['q2_len'].mean())

In [ ]:
# Analysis of features - q1_len_w
sns.displot(df['q1_len_w'])
print("max of q1_len_w",df['q1_len_w'].max())
print("min of q1_len_w",df['q1_len_w'].min())
print("mean of q1_len_W",df['q1_len_w'].mean())

In [ ]:
# Analysis of features - q2_len_w
sns.displot(df['q2_len_w'])
print("max of q2_len_w",df['q2_len_w'].max())
print("min of q2_len_w",df['q2_len_w'].min())
print("mean of q2_len_W",df['q2_len_w'].mean())

In [ ]:
# common words
sns.histplot(df[df['is_duplicate'] == 0]['word_common'], label='non_duplicate') # can use distplot too
sns.histplot(df[df['is_duplicate']== 1]['word_common'] , label = 'duplicate')
plt.legend()
plt.show()


In [ ]:
# total words
sns.histplot(df[df['is_duplicate'] == 0]['total_words'],label='non duplicate') # distplot is prettier
sns.histplot(df[df['is_duplicate'] == 1]['total_words'],label='duplicate')
plt.legend()
plt.show()

In [ ]:
# word share
sns.histplot(df[df['is_duplicate'] == 0]['word_share'],label='non duplicate') #dist
sns.histplot(df[df['is_duplicate'] == 1]['word_share'],label='duplicate')
plt.legend()
plt.show()

In [ ]:
# apply bow to q1 n q2.
# Considering is_duplicate column as y.
# apply random forest algorithm

ques_df = df[['question1','question2']]
ques_df.head()
final_df = df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)
final_df.head()

In [ ]:
# to find bow use count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge to make a list of all questions
# then use countvectorizer to get features
questions = list(ques_new_df['question1']) + list(ques_new_df['question2']) #getting all the questions
cv = CountVectorizer(max_features=3000) # limit num of features
# now use these features to get bow for all ques
# split this list of bow of all ques into 2 sets again -> q1 and q2
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2) # transform to get bow

In [ ]:
# convert to df n concatenate
temp_df1 = pd.DataFrame(q1_arr, index = ques_new_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_new_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape # sparse array

In [ ]:
final_df = pd.concat([final_df, temp_df], axis=1)
print(final_df.shape)
final_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
# splitting data
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state =1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# applying random forest classifier
rf = RandomForestClassifier() # the model
rf.fit(X_train, y_train)    # train
y_pred = rf.predict(X_test)  # test make predicttions on test data
accuracy_score(y_test,  y_pred)    # check for accuracy

In [ ]:
from xgboost import XGBClassifier
# checking accuracy with xgboost
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test, y_pred)